In [1]:
import pandas as pd
import numpy as np
import json
 
from statsmodels.api import OLS, add_constant
import pandas_datareader.data as web
import yfinance as yf
from sklearn.linear_model import LinearRegression

In [2]:
# Define default parameters
tickers = ['AAPL', 'MSFT', 'TSLA', 'META', 'AMZN']
start_date = '2000-01-01'
end_date = '2023-05-02'

# For training and determining the signal to trade
testing_date = '2023-03-01'
decision_threshold = 0.001
transaction_amount = 1

In [3]:
# Getting the fama-french research results
ff_factor = 'F-F_Research_Data_5_Factors_2x3_daily'
ff_factor_data = web.DataReader(ff_factor, 'famafrench', start=start_date, end=end_date)[0]
ff_factor_data = ff_factor_data.div(100)

ff_factor_data.tail()

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
2023-03-27,0.0027,0.0051,0.0102,-0.0028,0.0035,0.00016
2023-03-28,-0.0017,-0.0003,0.0074,0.0008,0.0055,0.00016
2023-03-29,0.0139,-0.0034,-0.0050,-0.0090,-0.0054,0.00016
2023-03-30,0.0051,-0.0061,-0.0059,0.0020,-0.0009,0.00016
2023-03-31,0.0153,0.0051,-0.0077,-0.0046,-0.0076,0.00016


In [4]:
# Getting the stock price data
ff_data = ff_factor_data

for ticker in tickers:
  ticker_data = yf.download(ticker, start_date, end_date, interval='1d')
  daily_return = ticker_data['Adj Close'].pct_change().dropna()
  
  # Calculate the excess return
  excess_return = daily_return.sub(ff_factor_data['RF'], axis=0)
  excess_return.name = ticker + '_RT'

  # Merge to the collection
  ff_data = ff_data.merge(excess_return, on='Date')

ff_data.dropna(inplace=True)
ff_data.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Mkt-RF,SMB,HML,RMW,CMA,RF,AAPL_RT,MSFT_RT,TSLA_RT,META_RT,AMZN_RT
Date,,,,,,,,,,,
2012-05-21,0.0169,0.0055,-0.0110,-0.0029,-0.0063,0.0,0.058260,0.016399,0.043904,-0.109861,0.019921
2012-05-22,0.0000,-0.0085,0.0035,-0.0011,-0.0001,0.0,-0.007678,0.000336,0.070559,-0.089039,-0.012746
2012-05-23,0.0026,0.0050,-0.0023,-0.0004,-0.0007,0.0,0.024400,-0.021842,0.007143,0.032258,0.009056
2012-05-24,0.0016,0.0002,0.0018,0.0036,0.0054,0.0,-0.009184,-0.001374,-0.023855,0.032187,-0.009389
2012-05-25,-0.0016,0.0024,-0.0014,0.0018,0.0013,0.0,-0.005360,-0.000344,-0.015522,-0.033909,-0.010918


In [5]:
# Define our X and Y variables
X = ff_data[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
Y = ff_data.drop(columns={'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF'}, axis=1)

In [6]:
# Split into testing/training data
X_train = X[X.index < testing_date]
X_test = X[X.index >= testing_date]
Y_train = Y[Y.index < testing_date]
Y_test = Y[Y.index >= testing_date]

In [7]:
# Using the linear model from Sklearn
lr_model = LinearRegression(fit_intercept=True)
lr_model = lr_model.fit(X_train, Y_train)
predictions = lr_model.predict(X_test)
predictions = predictions.transpose()

In [8]:
# Do comparisons betweeen the prediction and testing data
results = Y_test.copy()

for ind in range(len(tickers)): 
  results[tickers[ind] + '_Pred'] = predictions[ind]
  
results.head()

,AAPL_RT,MSFT_RT,TSLA_RT,META_RT,AMZN_RT,AAPL_Pred,MSFT_Pred,TSLA_Pred,META_Pred,AMZN_Pred
Date,,,,,,,,,,
2023-03-01,-0.014406,-0.012789,-0.014452,-0.008849,-0.022021,-0.007109,-0.009736,-0.011776,-0.012690,-0.013817
2023-03-02,0.003969,0.019493,-0.058699,0.006241,-0.000594,0.010756,0.010502,0.009790,0.007757,0.008444
2023-03-03,0.034930,0.016486,0.035932,0.061262,0.029906,0.019675,0.022438,0.032889,0.026289,0.026296
2023-03-06,0.018379,0.006029,-0.020282,-0.002049,-0.012278,0.003967,0.007316,-0.006481,0.002005,0.005816
2023-03-07,-0.014656,-0.010749,-0.031634,-0.002269,-0.002293,-0.015471,-0.016953,-0.008812,-0.012152,-0.012045


In [9]:
# Computing the rolling weekly average
window = 5
rolling_w_avg = results.rolling(window).mean().dropna()

# Construct the signal to buy if rolling weekly of prediction is higher than returns by the decision threshold
# sell if vice versa
signals = rolling_w_avg.copy()
decision = pd.DataFrame(index=signals.index)

for ticker in tickers:
  
  signals[ticker + '_Buy'] = np.where(signals[ticker + '_Pred'] > signals[ticker + '_RT'] + decision_threshold, 1.0 * transaction_amount, 0.0)
  signals[ticker + '_Sell'] = np.where(signals[ticker + '_Pred'] < signals[ticker +'_RT'] - decision_threshold, -1.0 * transaction_amount, 0.0)
  
  decision[ticker] = signals[ticker + '_Buy'] + signals[ticker + '_Sell']

signals.tail()


,AAPL_RT,MSFT_RT,TSLA_RT,META_RT,AMZN_RT,AAPL_Pred,MSFT_Pred,TSLA_Pred,META_Pred,AMZN_Pred,AAPL_Buy,AAPL_Sell,MSFT_Buy,MSFT_Sell,TSLA_Buy,TSLA_Sell,META_Buy,META_Sell,AMZN_Buy,AMZN_Sell
Date,,,,,,,,,,,,,,,,,,,,
2023-03-27,0.001004,0.002944,0.009678,0.005004,0.000641,0.002184,0.003016,0.003794,0.003502,0.003851,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,1.0,0.0
2023-03-28,-0.002180,0.000973,-0.008694,-0.001524,-0.006927,-0.001040,-0.001545,-0.004182,-0.003339,-0.002894,1.0,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0,0.0
2023-03-29,0.003598,0.005898,0.002773,0.005455,0.003061,0.004225,0.004853,0.008027,0.004995,0.004847,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,1.0,0.0
2023-03-30,0.004183,0.004478,0.003098,0.003406,0.006532,0.004276,0.004632,0.006245,0.003561,0.003238,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0
2023-03-31,0.005650,0.005374,0.017455,0.005657,0.010236,0.007147,0.008125,0.013873,0.009166,0.008839,1.0,0.0,1.0,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0


In [10]:
decision.reset_index(inplace=True)
decision['Date'] = decision['Date'].astype(str)
decision.tail()

,Date,AAPL,MSFT,TSLA,META,AMZN
14,2023-03-27,1.0,0.0,-1.0,-1.0,1.0
15,2023-03-28,1.0,-1.0,1.0,-1.0,1.0
16,2023-03-29,0.0,-1.0,1.0,0.0,1.0
17,2023-03-30,0.0,0.0,1.0,0.0,-1.0
18,2023-03-31,1.0,1.0,-1.0,1.0,-1.0


In [11]:
# Convert the decisions to JSON
out_file = 'fama_french.json'

info = {'tickers' : ticker,
        'start_date' : testing_date,
        'end_date' : end_date}

with open(out_file, mode='w') as f:
    f.write(json.dumps(info, indent=2))
    f.write('\n')

# Append the decision information
with open(out_file, mode="a") as f:
    f.write(decision.to_json(orient="records", lines=True))